In [1]:
import mmcv
import torch
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)
from mmcv.utils import DictAction
from mmcv import Config
from mmseg.apis import multi_gpu_test, single_gpu_test
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from glob import glob

import matplotlib.pyplot as plt
import albumentations as A
from mmcv import Config
import os
import numpy as np
from tqdm import tqdm
import pandas as pd

In [2]:
MODEL = 'OCR_swinB_aux.py' # model config 경로
PATH = '/opt/ml/segmentation/moon/mmsegmentation/work_dirs/OCR_swinB_aux' # 모델 저장된 폴더
BEST_CHECKPOINT = glob(os.path.join(PATH,'best_*'))
assert len(BEST_CHECKPOINT)==1
BEST_CHECKPOINT = BEST_CHECKPOINT[0]
# EPOCH ='best_mIoU_epoch_39.pth' # model checkpoint 파일이름

cfg =Config.fromfile(os.path.join(PATH,MODEL))
cfg.data.test.test_mode = True

In [3]:
size_min = 512
size_max = 1024
multi_scale_list = [(x,x) for x in range(size_min, size_max+1, 128)]
cfg.data.test.pipeline[1]['img_scale'] = multi_scale_list # Resize
cfg.data.test.pipeline[1]['flip']=True
cfg.data.test.img_dir = 'images/test'
cfg.data.test.ann_dir = None

In [4]:
test_dataset = build_dataset(cfg.data.test)
test_loader = build_dataloader(
        test_dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2021-10-20 13:30:32,309 - mmseg - INFO - Loaded 819 images


In [5]:
# build model
cfg.model.pretrained = None
cfg.model.train_cfg = None

checkpoint_path = BEST_CHECKPOINT

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

Use load_from_local loader


In [6]:
model.CLASSES = checkpoint['meta']['CLASSES']
model.PALETTE = checkpoint['meta']['PALETTE']
model = MMDataParallel(model.cuda(), device_ids=[0])

In [7]:
output = single_gpu_test(model, test_loader)

[                               ] 12/819, 0.3 task/s, elapsed: 37s, ETA:  2492s

In [10]:
size = 256
transform = A.Compose([A.Resize(size, size)])

file_name_list=[]
preds_array = np.empty((0,size*size),dtype=np.int64)
img_infos = test_dataset.img_infos

for idx,out in tqdm(enumerate(output)):
    image = np.zeros((1,size,size))
    transformed = transform(image=image,mask=out)
    mask = transformed['mask']
    
    mask = mask.reshape(-1,size*size).astype(int)
    preds_array = np.vstack((preds_array,mask))
    
    file_name_list.append(img_infos[idx]['filename'])
    

819it [05:53,  2.31it/s]


In [11]:
# sample_submisson.csv 열기
tmp_submission = pd.read_csv('/opt/ml/segmentation/baseline_code/submission/sample_submission.csv', index_col=None)

# PredictionString 대입
for file_name, string in zip(file_name_list, preds_array):
    file_name = '/'.join(file_name.split('+'))
    tmp_submission = tmp_submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장

# 순서도 같아야 채점이 되기 때문에 sample_submission이 필요
sample = pd.read_csv('/opt/ml/segmentation/moon/submission/sample_submission.csv', index_col=None)
submission = pd.read_csv('/opt/ml/segmentation/baseline_code/submission/sample_submission.csv', index_col=None)
for image_id in sample['image_id'].tolist():
    prediction_string = tmp_submission[tmp_submission['image_id']==image_id]['PredictionString'].iloc[0]
    submission = submission.append({"image_id" : image_id, "PredictionString" : prediction_string }, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(f"./submission/{MODEL.split('.')[0]}_TTA.csv", index=False)


---

# Pseudo Labeling
- submission을 기반으로 할 경우, 256x256을 upssampling해야하기 때문에 output을 기반으로 pseudo labeling

In [27]:
import cv2

In [30]:
pseudo_path = '/opt/ml/segmentation/moon/dataset/annotations/pseudo/'
img_infos = test_dataset.img_infos
file_names = [img_info['filename'] for img_info in img_infos]
for file_name,img in zip(file_names,output):
    file_path = os.path.join(pseudo_path,file_name)
    cv2.imwrite(file_path,img)

---